t# Minimos quadrados recursivo

O modelo do sistema é atualizado a cada periodo de amostragem, quando novas amostras estão disponíveis.

## Deduções

Thetas são os a e b, e tem que ter uma realimentação dos dados passados em razão a isso. Por isso o Theta vai estar sempre um instante atrasado. Existe um mecanismo de adaptação para corrigir as estimativas, os a's e b's. Só fazer um pequeno ajuste no modelo, ao invés de recalcular todo o modelo.

Duas caracteristicas importantes: Polarização (bias) e covariancia.

Se o erro tiver uma média zero com uma certa variancia ele é ruído branco, caso ele tenha uma média não zero o ruído é colorido.

Teste lyung vai verificar os residuos (média zero ou não, e outros residuos).

### Formulas
$$\hat{\theta}(t) = [\theta^T\theta]^{-1}\theta^T Y$$

### Inversa

Atualizar a inversa sem calcular a matrix inversa, economizando processamento.

P(t) -> Matrix de covariância (Muito importante). Qual a influência de uma variavel em relação as outras.

$$\hat{\theta}(t+1) = \hat{\theta}(t) + P(t+1)\theta(t+1)e(t+1)$$
$$\hat{\theta}(t+1) = \hat{\theta}(t) + K(t+1)e(t+1)$$

K é o ganho de Kalman (filtro de Kalman), é o ganho do estimador que se multiplica pelo erro.

Rank da Matrix P: Soma dos elementos da diagonal principal, sendo a matrix a covariancia entre os a's e b's. O objetivo é que os valores sejam pequenos.

O valor nos valores na matrix P implica o quanto eu confio nos valores de a's e b's, quanto menor maior a confiança nos valores gerados. Isso altera a dinamica de busca dos parametros.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb

print('Estimador dos mínimos quadrados recursivo aplicado a uma planta de segunda ordem\n\n')

nit  = input('Quantas iterações de simulação deseja (> 4) ? ')

u = np.sign(2*np.random.normal(1,nit)-1) # sinal de entrada PRBS (Pseudo Random Binary Sequence)

e     = u*0.01;         # gera um ruído a ser inserido na função de transferênca
P     = 1000*np.eye(4,4);  # inicia a matriz de covariância co ordem: (na+nb) x (na+nb)
theta = [0]*4;      # condições iniciais dos parâmetros estimados

# inicia as variáveis
inicia = 3
y      = np.zeros(1,inicia)
erro   = np.zeros(1,inicia)
a1     = np.zeros(1,inicia)
a2     = np.zeros(1,inicia)
b0     = np.zeros(1,inicia)
b1     = np.zeros(1,inicia)
traco  = np.zeros(1,inicia)
ganho = []
# iteração da simulação
for t in range(3,nit):
    # função de transferência do processo (equação diferenças) 
    y[t] = 1.5144*y(t-1) - 0.5506*y(t-2) + 0.599*u(t-1)  + 0.163*u(t-2)  + e(t);
   
    # vetor de medidas (processo de 2a. ordem)
    fi = [ -y(t-1), -y(t-2), u(t-1), u(t-2) ]
   
    # calcula o erro de estimação (saída real - saída estimada)
    erro[t] = y(t) - theta.T*fi
   
    # calcula vetor de ganho
    K = P*fi / (1 + fi.T*P*fi);          
   
    # atualiza vetor de parâmetros
    theta = theta + K*erro(t);          
   
    # atualiza matriz de covariância
    P = P - K*fi.T*P;                 

    a1[t] = theta(1)
    a2[t] = theta(2)
    b0[t] = theta(3)
    b1[t] = theta(4)

    traco[t] = np.trace(P)
    ganho.append(sum(abs(K)))

   # traço da matriz de covariância (soma dos elementos da diag. princ.)
   # fprintf('iteração %d/%d\t traço de P:%f\n', t,nit,traco(t));

# gráfico dos parâmetros obtidos
t = list(range(1,nit+1))
#subplot(231)
plt.plot(t,a1[t],'k')
plt.ylabel('a_1')
plt.xlabel('amostra')

#subplot(232)
plt.plot(t,a2[t],'k')
plt.ylabel('a_2')
plt.xlabel('amostra')

#subplot(233)
plt.plot(t,b0[t],'k')
plt.ylabel('b_0')
plt.xlabel('amostra')

#subplot(234)
plt.plot(t,b1[t],'k')
plt.ylabel('b_1')
plt.xlabel('amostra')

#subplot(235)
plt.plot(t,traco[t],'k')
plt.ylabel('traço da matriz de covariância P')
plt.xlabel('amostra')

#subplot(236)
plt.plot(t,ganho[t],'k')
plt.ylabel('ganho K')
plt.xlabel('amostra')

print('\nParâmetros estimados ao final de %d amostras:\n\n',nit);
for i, t in enumerate(theta):
    print('theta(%d) = %f\n',i, t);
print('\ntraço final = %f\n', traco[-1])
print('ganho final = %f\n', ganho[-1])

Estimador dos mínimos quadrados recursivo aplicado a uma planta de segunda ordem


